In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=False
yu.slName='plots_notes_NST_allRatios'

tensorQ=False

In [ ]:
ensLs=['24','48']
# ensLs=['48']
L2ens={'24':'cA211.530.24','48':'cA2.09.48'}

ensembles=[L2ens[ensL] for ensL in ensLs]

ens2data={}
for ens in ensembles:
    if tensorQ:
        path={
            'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA211.530.24_tensor.h5',
            'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_tensor.h5',
            }[ens]
        yu.gjList=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt','sgmyz','sgmzx','sgmxy','sgmtx','sgmty','sgmtz']
    else:
        path={
            'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_a_cA211.530.24.h5',
            'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_c_cA2.09.48.h5',
            }[ens]
        yu.gjList=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt']
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and conjugated 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'r2pt': True, # making 2ptMat real for the rest frame # 'll2pt' has to be real for this flag
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        if ops[0].split(';')[1]=='0,0,0' and flags['r2pt']:
            t=np.real(t)
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
        
    return res

pt0='0,0,0'; ofs0=[('a','p'),('N1pi1,a','12'),]; ops0=yu.getops(pt0,'l1',ofs0)
pt1='0,0,1'; ofs1=[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]; ops1=yu.getops(pt1,'l1',ofs1)

L2T_ens={'24':'cA211.530.24','48':'cA2.09.48'}

L2ops={'0':ops0,'1':ops1}
L2ptofs={'0':(pt0,ofs0),'1':(pt1,ofs1)}
L2T_pt={'0':r'$\vec{p}=(0,0,0)$','1':r'$\vec{p}=(0,0,1)$'}

L2diags={'noj':yu.diags_jLoopless,'all':yu.diags_all}
L2T_diags={'noj':'no j-loop','all':'all diags'}
diagsLs=['noj','all']

In [ ]:
dtt0=4

# yu.sl_reset()
vfunc=lambda x:np.abs(x)
# vfunc=lambda x:np.abs(np.real(x))
# vfunc=lambda x:np.abs(np.imag(x))

x2ylims={
    ('24','0'):([0,0.6],[0,0.5]),
    ('24','1'):([0,0.1],[0,0.1]),
    ('48','0'):([0,0.06],[0,0.15]),
    ('48','1'):([0,0.02],[0,0.03]),
}

x2fitmax={}
x2fitmins={}

rela_threshold=0.2
fitminmin=7

try:
    x2fitmin
except:
    x2fitmin={}
    
x2fitcov={}

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    xUnit=lat.a
    fig, axs = yu.getFigAxs(4,2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [2, 1]})
    
    for i_opsL, opsL in enumerate(['0','1']):
        pt,ofs=L2ptofs[opsL]
        Nop=len(ofs)
        
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops),data['VEV']['pi0f']]
        
        def func(dat):
            t=yu.meanDeep(dat)
            t2pt,tVEVpi0f=t
            t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
            eVecs=yu.renormalize_eVecs(eVecs)
            t=eVecs[:,0,:].T
            t=t/t[0,:]
            eVecs_inv_tin=np.linalg.inv(eVecs)
            W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
            tt=np.concatenate([[W],t])
            return vfunc(tt)
        
        mean,err,cov=yu.jackknife(dat,func,sl_key=('GEVP_results',ensL,opsL),sl_saveQ=True)
        
        if (ensL,opsL) not in x2fitmax:
            fitmax=len(mean[0])-1
            flag=False
            for t in range(1,fitmax+1):
                for ind in range(len(mean)):
                    rela=np.abs(err[ind][t]/mean[ind][t])
                    if rela>rela_threshold:
                        fitmax=t-1
                        flag=True
                        break
                if flag:
                    break
            x2fitmax[(ensL,opsL)]=fitmax
            x2fitmins[(ensL,opsL)]=range(fitminmin,fitmax)
            
        
        irow=2*i_opsL+0;icol=0
        for iop in range(1,Nop):
            ind=iop+1
            tmin=dtt0+1; tmax=len(mean[ind]); tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            if opsL is '0':
                label=r'$i=N(1)\pi(1)$'
            else:
                label=r'$i=N(1)\pi(0)$' if iop is 1 else r'$i=N(0)\pi(1)$'
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white',label=label)

            tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white')
            
            if (ensL,opsL) in x2fitmin:
                tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop])
            
        ylim=x2ylims[(ensL,opsL)][0]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        irow=2*i_opsL+1;icol=0
        ind=0
        tmin=dtt0+1; tmax=len(mean[ind]);  tmax=x2fitmax[(ensL,opsL)]+1
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        if (ensL,opsL) in x2fitmin:
            tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0])
        ylim=x2ylims[(ensL,opsL)][1]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        fits=[]
        fitmax=x2fitmax[(ensL,opsL)]
        for fitmin in x2fitmins[(ensL,opsL)]:
            # print(fitmin,fitmax)
            fitsizeunit=fitmax-fitmin+1
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
                eVecs=yu.renormalize_eVecs(eVecs)
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                tt=np.concatenate([[W],t[1:]])[:,fitmin:fitmax+1]
                if opsL=='0':
                    return np.real(tt)
                return np.concatenate([np.real(tt),np.imag(tt)]) 
            
            if (ensL,opsL) in x2fitmin and fitmin==x2fitmin[(ensL,opsL)]:
                t_ops=ops.copy()
                ops=yu.getops(pt,'l1',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov1=yu.jackknife(dat2,func,outputFlatten=True)
                ops=yu.getops(pt,'l2',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov2=yu.jackknife(dat2,func,outputFlatten=True)
                x2fitcov[(ensL,opsL)]=(cov1,cov2)
                ops=t_ops

            if opsL=='0':
                def fitfunc(Wr,v1r):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit] 
            elif opsL=='1':
                def fitfunc(Wr,v1r,v2r,Wi,v1i,v2i):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit, [v2r]*fitsizeunit, [Wi]*fitsizeunit, [v1i]*fitsizeunit, [v2i]*fitsizeunit]
            
            res=yu.fit(dat,func,fitfunc,sl_key=('GEVP_fit',ensL,opsL,fitmin),sl_saveQ=True)
            pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,_=res
            
            # print(ensL,opsL)
            # print(func(dat))
            # print(pars_mean)
            
            if opsL=='0':
                pars_mean_C=pars_mean
                pars_err_C=pars_err
            else:
                pars_mean_C=pars_mean[:Nop]+1j*pars_mean[Nop:]
                pars_err_C=pars_err[:Nop]+1j*pars_err[Nop:]
            fits.append((pars_mean_C,pars_err_C,chi2R_mean,Ndof))
            
        pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
        imfit=np.argmax(probs)
        x2fitmin[(ensL,opsL)]=x2fitmins[(ensL,opsL)][imfit]
        
        for ifit,fitmin in enumerate(x2fitmins[(ensL,opsL)]): 
            mfc='white' if ifit!=imfit else None
            pars_mean_C,pars_err_C,chi2R_mean,_=fits[ifit]
            irow=2*i_opsL+0; icol=1
            for iop in range(1,Nop):
                tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
                axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
                ylim=x2ylims[(ensL,opsL)][0]
                shift_chi2=(ylim[1]-ylim[0])/15
                axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

            irow=2*i_opsL+1; icol=1
            iop=0
            tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
            axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
            ylim=x2ylims[(ensL,opsL)][1]
            shift_chi2=(ylim[1]-ylim[0])/15
            axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

        axs[2*i_opsL+1,1].set_xlim([0.55,1.25] if ensL is '24' else [0.55,1.4])
        
        # axs[2*i_opsL+0,0].text(.55, .95, L2T_pt[opsL], ha='center', va='top', size=30, transform=axs[2*i_opsL+0,0].transAxes)
        axs[2*i_opsL+1,0].text(.05, .95, L2T_pt[opsL], ha='left', va='top', size=30, transform=axs[2*i_opsL+1,0].transAxes)
        
        
    axs[2*1+1,0].set_xlabel(r'$t$ [fm]',fontsize=40)
    axs[2*1+1,1].set_xlabel(r'$t_{\mathrm{min}}$ [fm]',fontsize=40)
    axs[0,0].set_ylabel(r'$|v_{N,O_{i}}/v_{N,O_{N(0)}}|$',fontsize=40)
    axs[1,0].set_ylabel(r'$|W|}$',fontsize=40)
    axs[2,0].set_ylabel(r'$|v_{N,O_{i}}/v_{N,O_{N(1)}}|$',fontsize=40)
    axs[3,0].set_ylabel(r'$|W|}$',fontsize=40)
        
    
    # if ensL is '48':
    #     axs[2,0].set_yticks([0,0.006,0.012,0.018])
    
    if ensL is '48':
        axs[3,1].set_xticks([0.7,1.0,1.3])
    else:
        axs[3,1].set_xticks([0.7,1.0])
        
    axs[0,0].legend(loc='upper left')
    axs[2,0].legend(loc='upper left')
    # if ensL is '24':
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
    # else:
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
        
    for irow in range(4):
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=30)
        
    plt.tight_layout()
    # plt.savefig('fig/2ptGEVP_'+ensL+'.pdf')
    plt.close()
     
for ensL in ensLs:
    run(ensL)
    run(ensL)
    print(ensL)
        # break
    # break

print()    
print(x2fitmin)
print(x2fitmax)

In [ ]:
# utitlies

datmins=range(1,5+1)

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc)) \
    if a00!=0 else np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))

flags['dt2pt']=False
flags['dt3pt']=True

simone={
    'GA_0':[1.245,0.031],
    'GA_1':[1.129,0.027],
    'GPb_1':[48.9,2.8], 'GP_1':[45.2,2.6],
    'G5b_1':[12.4,0.5], 'G5_1':[62.2,0.3],
    'rPCAC_1':[1.06,0.05],
}

gA_exp=1.27641

xlim_rainbow=[-0.6,0.6]
xticks_rainbow=[-0.4,-0.2,0,0.2,0.4]
xticks2_rainbow=[-0.4,0,0.4]
xlabel_rainbow=r'$t_{\mathrm{ins}}-t_{\mathrm{s}}/2$ [fm]'
xlim_fit=[0,0.6]
xticks_fit=[0.1,0.3,0.5]
xlabel_fit=r'$t_{\mathrm{ins,min}}$ [fm]'

def eVecs2eVecW(eVecs_ltni,ensL,opsL):
    if eVecs_ltni.shape[-1]==1:
        return (np.zeros((len(eVecs_ltni),1))+1,np.zeros((len(eVecs_ltni))+1))
    
    W_l=[]; eVec_li=[]
    eVecs_ltni=yu.renormalize_eVecs(eVecs_ltni)
    for l in range(len(eVecs_ltni)):
        eVecs=eVecs_ltni[l]
        fitsizeunit=len(eVecs)
        t=eVecs[:,0,:].T
        t=t/t[0,:]
        eVecs_inv_tin=np.linalg.inv(eVecs)
        W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
        tt=np.concatenate([[W],t[1:]])
        if opsL=='0':
            t=np.real(tt)
            def fitfunc(Wr,v1r):
                return [[Wr]*fitsizeunit, [v1r]*fitsizeunit]
            Wr,v1r=yu.fit0(fitfunc,t,x2fitcov[(ensL,opsL)][l])[0]
            W_l.append(Wr); eVec_li.append([1,v1r])
        else:
            t=np.concatenate([np.real(tt),np.imag(tt)]) 
            def fitfunc(Wr,v1r,v2r,Wi,v1i,v2i):
                return [[Wr]*fitsizeunit, [v1r]*fitsizeunit, [v2r]*fitsizeunit, [Wi]*fitsizeunit, [v1i]*fitsizeunit, [v2i]*fitsizeunit]
            Wr,v1r,v2r,Wi,v1i,v2i=yu.fit0(fitfunc,t,x2fitcov[(ensL,opsL)][l])[0]
            W_l.append(Wr+1j*Wi); eVec_li.append([1,v1r+1j*v1i,v2r+1j*v2i])
    W_l=np.array(W_l); eVec_li=np.array(eVec_li)   
    return (eVec_li,W_l)

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype='Id'):
    (eVeca_li,Wa_l)=eVecs2eVecW(eVecsa_ltni,ensL,opasL)
    (eVecb_li,Wb_l)=eVecs2eVecW(eVecsb_ltni,ensL,opbsL)
    
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    if Itype in ['Id','Ip']:
        dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt'] and Itype in ['Id']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

proj2mat={
    'P0':np.array([[1,0],[0,1]])/2,
    'Px':np.array([[0,1],[1,0]])/2,
    'Py':np.array([[0,-1j],[1j,0]])/2,
    'Pz':np.array([[1,0],[0,-1]])/2,
}



In [ ]:
def FF2pips(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    mN=lat.getaEN(0); EN=lat.getaEN(1)
    p1=(2*math.pi)/(lat.L/lat.a)
    C=np.sqrt(2*EN*(EN+mN))
    Q2=p1**2-(EN-mN)**2
    
    ZS=lat.ZS_ns[0]; ZV=lat.ZV_ns[0]; ZP=lat.ZP_ns[0]; ZA=lat.ZA_ns[0]; ZT=lat.ZT_ns[0]
    
    t_RP_z_01=ZS*C*(-p1)**(-1)
    t_RAx_x_01=ZA*(-1)*C*(1j*(EN+mN))**(-1)
    t_RA4_z_01=ZA*(-1)*C*(p1*(EN-mN)/(2*mN))**(-1); t_RA4_z_01_1=t_RAx_x_01*(-1)*(-2*mN/(EN-mN))
    t_RAz_z_01=ZA*(-1)*C*(-1j*p1**2/(2*mN))**(-1); t_RAz_z_01_1=t_RAx_x_01*(-1)*(-2*mN*(EN+mN)/p1**2)
    t_PCAC_G5=(lat.amu/ZP)/mN; t_PCAC_GP=Q2/(4*mN**2)
    
    t_RTxy_z_01=ZT*C*(2j*mN*(EN+mN))**(-1)
    t_RTyz_x_01=ZT*C*(-1j*p1**2)**(-1); t_RTyz_x_01_1=t_RTxy_z_01*(-1)*(2*mN*(EN+mN)/(-p1**2))
    t_RT4x_y_01=ZT*C*(p1*(EN-mN))**(-1); t_RT4x_y_01_1=t_RTxy_z_01*(-1)*(-2*mN/(EN-mN))
    t_RT4z_0_01=ZT*C*(-2j*p1*(EN+mN)**(-1)); t_RT4z_0_01_1=(-1)*(mN/(EN+mN))
    
    pips={
        'RS_0_00':[(ZP,'id','P0')],
        'RS_0_11':[(ZP*(mN/EN)**(-1),'id','P0')],
        'RS_0_01':[(ZP*C*(EN+mN)**(-1),'id','P0')],
        
        'RV4_0_00':[(ZV,'gt','P0')],
        'RV4_0_11':[(ZV,'gt','P0')],
        # 'RVz_0_11':[(ZV*(-1j*p1*(EN+mN)/(2*EN**2))**(-1),'gz','P0')],
        'RVz_0_11':[(ZV*(-1j*p1/(EN))**(-1),'gz','P0')],
        'RV4_0_01':[(ZV*C*(EN+mN)**(-1),'gt','P0')],
        'RVz_0_01':[(ZV*C*(-1j*p1)**(-1),'gz','P0')],
        'RVy_x_01':[(ZV*C*(p1)**(-1)*(1/2),'gy','Px'),(ZV*C*(p1)**(-1)*(-1/2),'gx','Py')],
        
        'RAz_z_00':[(ZA*(-1)*(-1j),'g5gz','Pz')],
        'RA4_z_11':[(ZA*(-1)*(-p1/EN)**(-1),'g5gt','Pz')],
        'RAz_z_11':[(ZA*(-1)*(1j*(mN/EN+p1**2/(EN*(EN+mN))))**(-1),'g5gz','Pz')],
        'RAx_x_11':[(ZA*(-1)*(1j*mN/EN)**(-1),'g5gx','Px')],
        'RP_z_11':[(ZS,'g5','Pz')],
        
        'RP_z_01':[(t_RP_z_01,'g5','Pz')],
        'RAx_x_01':[(t_RAx_x_01/2,'g5gx','Px'),(t_RAx_x_01/2,'g5gy','Py')],
        'RA4_z_01':[(t_RA4_z_01,'g5gt','Pz'),(t_RA4_z_01_1/2,'g5gx','Px'),(t_RA4_z_01_1/2,'g5gy','Py')],
        'RAz_z_01':[(t_RAz_z_01,'g5gz','Pz'),(t_RAz_z_01_1/2,'g5gx','Px'),(t_RAz_z_01_1/2,'g5gy','Py')],
        'rPCAC4_z_01':[(t_RAx_x_01/2,'g5gx','Px'),(t_RAx_x_01/2,'g5gy','Py'),\
            (t_RP_z_01*t_PCAC_G5,'g5','Pz'),(t_RA4_z_01*t_PCAC_GP,'g5gt','Pz'),(t_RA4_z_01_1/2*t_PCAC_GP,'g5gx','Px'),(t_RA4_z_01_1/2*t_PCAC_GP,'g5gy','Py')],
        'rPCACz_z_01':[(t_RAx_x_01/2,'g5gx','Px'),(t_RAx_x_01/2,'g5gy','Py'),\
            (t_RP_z_01*t_PCAC_G5,'g5','Pz'),(t_RAz_z_01*t_PCAC_GP,'g5gz','Pz'),(t_RAz_z_01_1/2*t_PCAC_GP,'g5gx','Px'),(t_RAz_z_01_1/2*t_PCAC_GP,'g5gy','Py')],
        
        'RTxy_z_00':[(ZT*(-1j),'sgmxy','Pz')],
        'RTyz_x_11':[(ZT*(-1j)/2,'sgmyz','Px'),(ZT*(-1j)/2,'sgmzx','Py')],
        'RTxy_z_11':[(ZT*(-1j)*(1-p1**2/(EN*(EN+mN)))**(-1),'sgmxy','Pz')],
        'RT4x_y_11':[(ZT*(-p1/EN)**(-1)/2,'sgmtx','Py'),(ZT*(-p1/EN)**(-1)*(-1)/2,'sgmty','Px')],
        
        'RTxy_z_01':[(t_RTxy_z_01,'sgmxy','Pz')],
        'RTyz_x_01':[(t_RTyz_x_01/2,'sgmyz','Px'),(t_RTyz_x_01/2,'sgmzx','Py'),(t_RTyz_x_01_1,'sgmxy','Pz')],
        'RT4x_y_01':[(t_RT4x_y_01/2,'sgmtx','Py'),(t_RT4x_y_01*(-1)/2,'sgmty','Px'),(t_RT4x_y_01_1,'sgmxy','Pz')],
        'RT4z_0_01':[(t_RT4z_0_01,'sgmtz','P0'),\
            (t_RTxy_z_01*t_RT4z_0_01_1,'sgmxy','Pz'),\
                (t_RTyz_x_01/2*t_RT4z_0_01_1,'sgmyz','Px'),(t_RTyz_x_01/2*t_RT4z_0_01_1,'sgmzx','Py'),(t_RTyz_x_01_1*t_RT4z_0_01_1,'sgmxy','Pz')],
        
    }[FF]
    return pips

FFs_S=['RS_0_00','RS_0_11','RS_0_01']
FFs_V=['RV4_0_00','RV4_0_11','RVz_0_11','RV4_0_01','RVz_0_01','RVy_x_01']
FFs_AP0=['RAz_z_00','RA4_z_11','RAz_z_11','RAx_x_11','RP_z_11']
FFs_AP1=['RP_z_01','RAx_x_01','RA4_z_01','RAz_z_01','rPCAC4_z_01','rPCACz_z_01']
FFs_T0=['RTxy_z_00','RTyz_x_11','RTxy_z_11','RT4x_y_11']
FFs_T1=['RTxy_z_01','RTyz_x_01','RT4x_y_01','RT4z_0_01']

FFinPCAC=['rPCAC4_z_01','rPCACz_z_01']

def FF2label_Rbar(FF):
    if FF in FFinPCAC:
        if FF in ['rPCAC4_z_01']:
            return r'$\bar{R}^{\Gamma_z}_{\mathrm{PCAC}_4}(\vec{0},\vec{1}_z)$'
        if FF in ['rPCACz_z_01']:
            return r'$\bar{R}^{\Gamma_z}_{\mathrm{PCAC}_z}(\vec{0},\vec{1}_z)$'

    ratio,gamma,mom=FF.split('_')
    moma,momb=mom
    mom2str={'0':r'\vec{0}','1':r'\vec{1}_z'}
    return r'$\bar{{R}}^{{\Gamma_{{{p1}}}}}_{{{p2}{p3}}}({p4},{p5})$'.format(R='R',p1=gamma,p2=ratio[1],p3='' if len(ratio)==2 else '_{'+ratio[2:]+'}',p4=mom2str[moma],p5=mom2str[momb])


def FF2ylim(ensL,FF,j):
    if FF in ['RS_0_11']:
        return FF2ylim(ensL,'RS_0_00',j)
    if FF in ['RV4_0_11','RVz_0_11']:
        return FF2ylim(ensL,'RV4_0_00',j)
    if FF in ['RA4_z_11','RAz_z_11','RAx_x_11']:
        return FF2ylim(ensL,'RAz_z_00',j)
    if FF in ['RTyz_x_11','RTxy_z_11','RT4x_y_11']:
        return FF2ylim(ensL,'RTxy_z_00',j)
    
    ind={('24','j+'):0,('24','j-'):1,('48','j+'):2,('48','j-'):3}[(ensL,j)]
    dic={
        'RS_0_00':([3.5,7.5],[0.4,1.2],[3.1,10.9],[-0.6,1.4]),
        'RV4_0_00':([2.51,3.19],[0.85,1.15],[2.3,3.3],[0.7,1.35]),
        'RAz_z_00':([0.2,0.8],[0.5,1.6],[0.2,0.8],[0.5,1.6]),
        'RTxy_z_00':([0.51,0.89],[0.86,1.19],[0.21,0.89],[0.71,1.24]),
    }
    if FF in dic:
        return dic[FF][ind]
    return None


In [ ]:
ylimDic={
    ('24','GS+_00'):[3,9.5], ('24','GS+_11'):[3,9.5],
    ('24','GSb+_00'):[80,220], ('24','GSb+_11'):[80,220],
    ('24','GA_00_zz'):[0.55,1.7], ('24','GA_11_tz'):[0.5,1.6], ('24','GA_11_zz'):[0.5,1.6], ('24','GA_11_xx'):[0.5,1.6],
    ('24','J5_11'):[-1.3,1.3],
    ('24','G5_01'):[0,30], ('24','GA_01'):[0.81,0.99], ('24','GP_01_tz'):[-30,40], ('24','GP_01_zz'):[5,17],
    ('24','rPCAC_1_Ptz'):[-2,3], ('24','rPCAC_1_Pzz'):[0.2,1.5],

    ('48','GS+_00'):[4,12.5], ('48','GS+_11'):[4,12.5], 
    ('48','GSb+_00'):[15,60], ('48','GSb+_11'):[15,60],
    ('48','GA_00_zz'):[0.55,1.7], ('48','GA_11_tz'):[0.5,1.6], ('48','GA_11_zz'):[0.5,1.6], ('48','GA_11_xx'):[0.5,1.6],
    ('48','J5_11'):[-1.3,1.3],
    ('48','G5_01'):[0,85], ('48','GA_01'):[0.96,1.24], ('48','GP_01_tz'):[-120,150], ('48','GP_01_zz'):[10,55],
    ('48','rPCAC_1_Ptz'):[-2,3], ('48','rPCAC_1_Pzz'):[0.2,1.5],
}


In [ ]:
# ratio
def runR(ensL,FF,Itype='Id'):
    ens=L2ens[ensL]
    updateEns(ens)
    str_Itype='' if Itype=='Id' else Itype
    
    opasL,opbsL=FF.split('_')[-1]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for j in ['j+','j-']:
            diagsL='all' if FF not in [] else 'noj'
            diags=L2diags[diagsL]
            needsVEV = (FF,j) in [('RS_0_00','j+'),('RS_0_11','j+')] and diagsL in ['all']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+j+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['RS_0_00','RS_0_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype=Itype)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            mean,err,_=yu.jackknife(dat,func,sl_key=('ratio'+str_Itype,ensL,FF,j,case,diagsL))
            
    print(ensL,FF,end='\t') 

In [ ]:
colorMap_rainbow={('noGEVP',10):'skyblue',('noGEVP',12):'royalblue',('noGEVP',14):'b',('GEVP',10):'tomato',('GEVP',12):'indianred',('GEVP',14):'red'}
colorMap_rainbow={('noGEVP',10):'green',('noGEVP',12):'blue',('noGEVP',14):'purple',('GEVP',10):'yellow',('GEVP',12):'orange',('GEVP',14):'red'}

# plot: ratio
oneColClass=['J5_11']+FFinPCAC
def runP(ensL,FFs,label,Itype='Id'):
    ens=L2ens[ensL]
    updateEns(ens)
    str_Itype='' if Itype=='Id' else Itype
    
    fig, axs = yu.getFigAxs(len(FFs),2,Lrow=4.8*1.2,Lcol=6.4*1.6,sharex=True)
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF.split('_')[-1]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        # ren=FF2ren(ensL,FF)
        
        for icol in [0,1]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks2_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=50)
        
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_Rbar(FF),fontsize=50)
        # axs[irow,1].set_ylabel(FF2label_charge[FF],fontsize=35)
        # axs[irow,1].yaxis.set_label_position("right")

        for icol,j in enumerate(['j+','j-']):
            diagsL='all'
            for case in ['noGEVP','GEVP']:
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                res=yu.sl(('ratio'+str_Itype,ensL,FF,j,case,diagsL))
                mean,err,cov=res
                
                if FF in FFinPCAC:
                    for i_tf,tf in enumerate(tfs):
                        tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf]])
                        tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                        # tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(1e-10)**2]])
                        tmean,tcov=yu.propagateError(lambda x:x[:tf-1]+x[tf-1:],tmean,tcov)
                        terr=np.sqrt(np.diag(tcov))
                        tMean=tmean;tErr=terr 
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                    continue
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    # tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  

    axs[0,0].set_title(r'$u+d$',fontsize=50)
    axs[0,1].set_title(r'$u-d$',fontsize=50)
    for irow in range(len(FFs)):
        for icol in [1]:
            axs[irow,icol].yaxis.tick_right()
            axs[irow,icol].yaxis.set_ticks_position('both')
        for icol in [0,1]:
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=50) 
            
    for irow,FF in enumerate(FFs):
        for icol,j in enumerate(['j+','j-']):
            if FF2ylim(ensL,FF,j) != None:
                axs[irow,icol].set_ylim(FF2ylim(ensL,FF,j))
            else:
                ylim=axs[irow,icol].get_ylim()
                mid=(ylim[0]+ylim[1])/2; diff=ylim[1]-mid
                ds=0.4
                ylim=[mid-diff*(1+ds),mid+diff*(1+ds)]
                axs[irow,icol].set_ylim(ylim)
                
    plt.tight_layout()
    plt.savefig('fig/ratio'+'_'+label+'_'+ensL+str_Itype+'.pdf')
    plt.close()
    print(ensL,label,end='\t') 


In [ ]:
Itype='Id'
for ensL in ensLs:
    if not tensorQ:
        for FF in FFs_S:
            runR(ensL,FF,Itype)
        for FF in FFs_V:
            runR(ensL,FF,Itype)
        for FF in FFs_AP0:
            runR(ensL,FF,Itype)
        for FF in FFs_AP1:
            runR(ensL,FF,Itype)
    else:
        for FF in FFs_T0:
            runR(ensL,FF,Itype)
        for FF in FFs_T1:
            runR(ensL,FF,Itype)
    print()

In [ ]:
for ensL in ensLs:
    if not tensorQ:
        runP(ensL,FFs_S,'S',Itype)
        runP(ensL,FFs_V,'V',Itype)
        runP(ensL,FFs_AP0,'AP0',Itype)
        runP(ensL,FFs_AP1,'AP1',Itype)
    else:
        runP(ensL,FFs_T0,'T0',Itype)
        runP(ensL,FFs_T1,'T1',Itype)
    print()

$R^\prime$

In [ ]:
# Itype='Ip'
# for ensL in ensLs:
#     if not tensorQ:
#         for FF in FFs_S:
#             runR(ensL,FF,Itype)
#         for FF in FFs_V:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP1:
#             runR(ensL,FF,Itype)
#     else:
#         for FF in FFs_T0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_T1:
#             runR(ensL,FF,Itype)
#     print()

In [ ]:
# for ensL in ensLs:
#     if not tensorQ:
#         runP(ensL,FFs_S,'S',Itype)
#         runP(ensL,FFs_V,'V',Itype)
#         runP(ensL,FFs_AP0,'AP0',Itype)
#         runP(ensL,FFs_AP1,'AP1',Itype)
#     else:
#         runP(ensL,FFs_T0,'T0',Itype)
#         runP(ensL,FFs_T1,'T1',Itype)
#     print()

$R^{\prime\prime}$

In [ ]:
# Itype='Ipp'

# if not tensorQ:
#     for ensL in ['48']:
#         for FF in FFs_S:
#             runR(ensL,FF,Itype)
#         for FF in FFs_V:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP1:
#             runR(ensL,FF,Itype)
#         runP(ensL,FFs_S,'S',Itype)
#         runP(ensL,FFs_V,'V',Itype)
#         runP(ensL,FFs_AP0,'AP0',Itype)
#         runP(ensL,FFs_AP1,'AP1',Itype)
#         print()
# else:
#     for ensL in ['24']:
#         for FF in FFs_S:
#             runR(ensL,FF,Itype)
#         for FF in FFs_V:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_AP1:
#             runR(ensL,FF,Itype)
#         for FF in FFs_T0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_T1:
#             runR(ensL,FF,Itype)
#         runP(ensL,FFs_S,'S',Itype)
#         runP(ensL,FFs_V,'V',Itype)
#         runP(ensL,FFs_AP0,'AP0',Itype)
#         runP(ensL,FFs_AP1,'AP1',Itype)
#         runP(ensL,FFs_T0,'T0',Itype)
#         runP(ensL,FFs_T1,'T1',Itype)
#         print()
#     for ensL in ['48']:
#         for FF in FFs_T0:
#             runR(ensL,FF,Itype)
#         for FF in FFs_T1:
#             runR(ensL,FF,Itype)
#         runP(ensL,FFs_T0,'T0',Itype)
#         runP(ensL,FFs_T1,'T1',Itype)
#         print()